This is going to be a very basic tutorial on how to use 
 1. [Julia](https://julialang.org/)
 2. [Quaternions.jl](https://juliageometry.github.io/Quaternions.jl/stable/)


### Aim:
Rotate a point $P=(1,0,0)$ about the $z-axis$ by $\frac{\pi}{2}$ radians using Quaternions.

<u>To cut the wild goose chase, Here's the very simple solution:</u>

---

```julia
using Quaternions

u = axis = [0, 0, 1] # rotation axis (z-axis)
theta = π/2 # rotation angle (90 degrees in radians)
q = axisangle2quat(u, theta) # 90 degree rotation about z-axis # rotation quaternion: cos(θ/2) + u*sin(θ/2)
# or q = Quaternion(cos(theta/2), (u*sin(theta/2))...)

v = [1, 0, 0] # vector to be rotated


v_rotated = imag_part(q * Quaternion(0, v...) * inv(q)) # rotate vector v using quaternion q
# This is saying (0, v_rotated) = q * (0, v) * q^{-1}, as discussed in the notes
# So note, the required vector, is the imaginary part of the resulting quaternion, not the resulting quaternion itself

println("Original Vector: \n\t", v)
println("Rotated Vector: \n\t", v_rotated)

```

---

My hope is, by the end of reading this notebook, this will be simple to you too!

$\,$

$\,$



## 1. Setup
The first step, which I trust you to have done, is install **Julia** correctly.
(On windows, preferably through MIcrosoft Store, for others, via the methods described in the [Julia Installation Page](https://julialang.org/install/))

In [ ]:
# If you havn't already, setup the environment and install the required packages
# This is just a practice that you should generally do at the start of a new Julia project
# I prefer to use shared environments for all my projects - you don't need to worry about this

# Using the Packages package to manage packages
using Pkg

# Activate a shared environment for this project
Pkg.activate("Rotations", shared=true)

# Install the Quaternions package - you can add other packages here too
Pkg.add(["Quaternions"]) # you only need to do this once

## 2. Importing the Package and Defining a Quaternion

In [ ]:
# Now you can use the packages you installed
using Quaternions 

# Define a quaternion

# A quaternion is defined as q = a + bi + cj + dk
# For more details on how we are going to use quaternions, check the Notes

# We would define the quaternion q = 1 + 0i + 0j + 0k
q = Quaternion(1, 0, 0, 0)

println(q)

## 3. Working with Quaternions

In [ ]:
# adding 2 quaternions
q1 = Quaternion(1, 2, 3, 4) # q1 = 1 + 2i + 3j + 4k
q2 = Quaternion(5, 6, 7, 8) # q2 = 5 + 6i + 7j + 8k
q3 = q1 + q2
println("q1 = $q1")
println("q2 = $q2")
println("q3 = q1 + q2 = $q3")

In [ ]:
# multiplying 2 quaternions
q1 = Quaternion(2, 0, 0, 0)
q2 = Quaternion(0, 1, 0, 1)
q3 = q1 * q2

println("q1 = $q1")
println("q2 = $q2")
println("q3 = q1 * q2 = $q3")

# We will learn more about quaternions as we go along

## Looking the tools we have around Quaternions as well as defining our own tooling

In [ ]:
# Print Quaternion as "a + bi + cj + dk"
# Defining a custom function to convert a quaternion to string

q = Quaternion(1, 2, 3, 4)

println(real(q)) # prints the real part of the quaternion
println(imag_part(q)) # prints the imaginary part of the quaternion as a 3D vector

function toString(q::Quaternion)
    # You can access the components of the quaternion using q.s, q.v1, q.v2, and q.v3 (scalar and vector parts)
    # In general, you can run `fieldnames(Quaternion)` to see all the fields of the Quaternion type if you ever forget this
    return "$(q.s) + $(q.v1)i + $(q.v2)j + $(q.v3)k"
end

println("String representation: \n", toString(q))

---

## A short refresher: imaginary numbers and Euler's formula

An imaginary unit $i$ satisfies $i^2 = -1$. Complex numbers $z = a + b i$ rotate the plane via Euler's formula:
$$
e^{i\theta} = \cos\theta + i\sin\theta.
$$
Multiplying by $e^{i\theta}$ rotates by angle $\theta$.

## Quaternions as an extension of complex rotations

A quaternion has scalar and vector parts:
$$
q = w + x\mathbf{i} + y\mathbf{j} + z\mathbf{k} \equiv (w, \mathbf{v}),\quad \mathbf{v} = (x, y, z).
$$
A pure quaternion has $w=0$; for any unit vector $\hat{u}$ (as a pure quaternion), $\hat{u}^2 = -1$.

Multiplication rules:
$$
\mathbf{i}^2 = \mathbf{j}^2 = \mathbf{k}^2 = \mathbf{ijk} = -1,
$$
and anti-commutation:
$$
\mathbf{i}\mathbf{j} = \mathbf{k},\quad \mathbf{j}\mathbf{i} = -\mathbf{k},\quad
\mathbf{j}\mathbf{k} = \mathbf{i},\quad \mathbf{k}\mathbf{j} = -\mathbf{i},\quad
\mathbf{k}\mathbf{i} = \mathbf{j},\quad \mathbf{i}\mathbf{k} = -\mathbf{j}.
$$

### Quaternion exponential

For any unit pure quaternion $\hat{u}$ and real $\theta$,
$$
e^{\hat{u}\theta} = \cos\theta + \hat{u}\sin\theta
$$
This generalizes Euler's formula. For now, think of $\hat{u}$ as a fixed imaginary direction.

## Using $e^{\hat{u}\theta}$ to rotate

Multiplying a pure quaternion vector by $e^{\hat{u}\theta}$ alone does not yield a pure quaternion. To rotate a 3D vector $\mathbf{v}$ (as $v = (0, \mathbf{v})$), use the sandwich product:
$$
q v q^{-1}
$$
where $q = e^{\hat{u}\alpha} = \cos\alpha + \hat{u}\sin\alpha$.

The key identity:
$$
q v q^{-1} = \left(0,\,
\mathbf{v}\cos(2\alpha) + (\hat{u}\times\mathbf{v})\sin(2\alpha)
+ \hat{u}(\hat{u}\cdot\mathbf{v})(1-\cos(2\alpha))
\right)
$$
This is Rodrigues' rotation formula: $q v q^{-1}$ rotates $\mathbf{v}$ by $2\alpha$ about $\hat{u}$. (Essentially both $q$ multiplications cause the vector to rotate by $alpha$ angle. First one gets it to the )

To rotate by angle $\theta$, set $\alpha = \theta/2$:
$$
q(\hat{u}, \theta) = e^{\hat{u}\theta/2} = \cos\frac{\theta}{2} + \hat{u}\sin\frac{\theta}{2}
$$
Then $q v q^{-1}$ rotates $\mathbf{v}$ by $\theta$ about $\hat{u}$. The half-angle arises because the sandwich doubles the rotation angle.

---

In [ ]:
axis = [0, 0, 1] # rotation axis (z-axis)
angle = π/2 # rotation angle (90 degrees in radians)

# Different ways to define a rotation quaternion: All these are equivalent
q_rot = Quaternion(cos(angle/2), (axis*sin(angle/2))...) # rotation quaternion: cos(θ/2) + u*sin(θ/2)
q_rot_euler = exp(Quaternion(0, (axis*angle/2)...)) # rotation quaternion using exp: e^(u*θ/2)
q_rot_euler2 = ℯ^(Quaternion(0, (axis)...) * angle/2) # rotation quaternion using exp (alternative syntax to the above)


# Confirming that all the above quaternions are equal
println("Rotation Quaternion using cos/sin: \n\t$(toString(q_rot))\n")
println("Rotation Quaternion using exp: \n\t$(toString(q_rot_euler))\n")
println("Rotation Quaternion using exp (alternative syntax): \n\t$(toString(q_rot_euler2))\n")

println("\nAre they equal? \n\t", q_rot == q_rot_euler == q_rot_euler2)

So we can see how we can define a Quaternion that rotates a vector about an axis for a given angle.
Now, let's see how to rotate vector/point $P=(1,0,0)$ using this quaternion.

For this, we also need a quaternion inverse, which as we discussed earlier, is the same as the conjugate.

$$
q * q^{-1} = q*inv(q) = q_0
$$
$q_0$ being your identity quaternion $1 + 0i + 0j + 0k$, meaning, a simple real number $1$, which makes sense, since an identity quaterion should not change any vector in any form: orentation or scale.

In [ ]:
# Finding the Inverse

# Define a function to convert axis-angle to quaternion from our established formula
axisangle2quat(axis, angle) = Quaternion(cos(angle/2), (axis*sin(angle/2))...)
# q(axis, angle) = (cos(θ/2), u*sin(θ/2)) = (w, xi, yj, zk)

axis = [0, 0, 1] # rotation axis (z-axis)
angle = π/2 # rotation angle (90 degrees in radians)
q = axisangle2quat(axis, angle) # rotation quaternion: cos(θ/2) + u*sin(θ/2)
println("Rotation Quaternion using axis-angle to quaternion function: \n\t$(toString(q))\n")


# Equivalent ways to find the inverse of a unit quaternion
q_inverse = inv(q)
q_conjugate = conj(q)
q_axis_negative_angle = axisangle2quat(axis, -angle)


# Confirming that all the above quaternions are equal
println("Quaternion Inverse: \n\t$(toString(q_inverse))\n")
println("Quaternion Conjugate: \n\t$(toString(q_conjugate))\n")
println("Quaternion for negative angle: \n\t$(toString(q_axis_negative_angle))\n")
println("\nAre they equal? \n\t", q_inverse == q_conjugate == q_axis_negative_angle)


q * q_inverse # should be identity quaternion

# Confirming that the product of a quaternion and its inverse is the identity quaternion
println("\n------------\nq * q_inverse = ", toString(q * q_inverse), " = Identity Quaternion") # should be identity quaternion

Finally, since we now have all the ingedients, we can rotate a vector
## The Final Implementation:

In [ ]:
using Quaternions

u = axis = [0, 0, 1] # rotation axis (z-axis)
theta = π/2 # rotation angle (90 degrees in radians)
q = axisangle2quat(u, theta) # 90 degree rotation about z-axis # rotation quaternion: cos(θ/2) + u*sin(θ/2)
# or q = Quaternion(cos(theta/2), (u*sin(theta/2))...)

v = [1, 0, 0] # vector to be rotated


v_rotated = imag_part(q * Quaternion(0, v...) * inv(q)) # rotate vector v using quaternion q
# This is saying (0, v_rotated) = q * (0, v) * q^{-1}, as discussed in the notes
# So note, the required vector, is the imaginary part of the resulting quaternion, not the resulting quaternion itself

println("Original Vector: \n\t", v)
println("Rotated Vector: \n\t", v_rotated)

You can see that the output vector that we get is basically $[0,1,0]$
(The error here is of the order of $10^{-16}$, but would always be there when dealing with Floating Point numbers. Below, you can see you get a similar error with rotational matrices. However, it is much simpler to define Quaternions for an arbitrary axis and angle and do inverses that matrices)

In [ ]:
# Comparison with rotation matrix
rotzmat(angle) = [
    cos(angle) -sin(angle) 0
    sin(angle)  cos(angle) 0
    0          0         1
]

rotzmat(pi/2) * v # should be [0, 1, 0]

---

## Conclusion

It makes sense!
We rotated a point $P=[1,0,0]$ (or say the $x-axis$), around the $z-axis$, by $\pi/2$ radians, and therefore, it lands on the $y-axis$. Therefore:

Our quaternion $q(axis, angle) = cos(\theta/2) + u\cdot sin(\theta/2)$ maps $[1,0,0] \longrightarrow [0,1,0]$ by the operation $(0, v_{rot} = q*v*q^{-1})$ successfully.


This intuition, of a rotation mapping a vector, and therefore axes themselves from one space to another would help us understand how local transformations are mapped to global transformations.